# 기본 패키지 import

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## selenium import 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# 스타벅스 데이터 추출하기 

스타벅스 데이터 추출하기



In [72]:
## 화면에 따른 element 나 dom 변경을 방지하기위해 화면 사이즈를 특정 사이즈로 고정

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
driver = webdriver.Chrome(options=options)


크롤링 페이지 : https://www.starbucks.co.kr/store/store_map.do?disp=locale


In [73]:
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

로딩 마커 확인 
- xpath : /html/body/div[@class='loading_dim']
- 해당 요소가 가려질때까지 대기

In [74]:


##

loading_check=driver.find_element(By.XPATH, '/html/body/div[@class="loading_dimm"]')
while loading_check.is_displayed():
    print(f' loading check... : style: {loading_check.get_attribute("style")}')
    time.sleep(0.1)
    loading_check=driver.find_element(By.XPATH, '/html/body/div[@class="loading_dimm"]')

 loading check... : style: z-index: 20000; display: block; opacity: 0.133;
 loading check... : style: z-index: 20000; display: block; opacity: 0.41328;
 loading check... : style: z-index: 20000; display: block; opacity: 0.616683;
 loading check... : style: z-index: 20000; display: block; opacity: 0.697882;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; display: block;
 loading check... : style: z-index: 20000; 

지역 화면 활성화 대기

In [75]:
local_xpath='//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[@class="loca_search"]'

while True:
    try:
        local=driver.find_element(By.XPATH, local_xpath)
        break
    except:
        print('retrying...')
        time.sleep(0.1)


목차 화면대기

In [76]:
area_select_xpath='//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'

while True:
    try:
        area_select=driver.find_element(By.XPATH, area_select_xpath)
        break
    except:
        print('retrying...')
        time.sleep(1)

area_select.click()

all_selct_xpath='//*[@id="mCSB_2_container"]/ul/li[1]/a'
while True:
    try:
        all_select=driver.find_element(By.XPATH, all_selct_xpath)
        break
    except:
        print('retrying...')
        time.sleep(1)

all_select.click()


retrying...
retrying...
retrying...
retrying...


In [77]:
items='//*[@id="mCSB_3_container"]/ul/li'

while True:
    try:
        item_list=driver.find_elements(By.XPATH, items)
        if len(item_list)>0:
            break
    except:
        print('retrying...')
        time.sleep(1)
len(item_list)

613

In [78]:
item_list[0].get_attribute('innerText').split('\n')
## 이름 : 0 , 주소 : 2, 전화번호 : 3

['역삼아레나빌딩', '', '서울특별시 강남구 언주로 425 (역삼동)', '1522-3232', '', '리저브 매장 2번']

In [79]:
rets=[]
for item in item_list:
    ret={}
    item_ele=item.get_attribute('innerText').split('\n')
    ret['name']=item_ele[0]
    ret['address']=item_ele[2]
    #ret['phone']=item_ele[3]
    ret['type']='starbucks'
    ret['lat']=item.get_attribute('data-lat')
    ret['lng']=item.get_attribute('data-long')
    rets.append(ret)

df=pd.DataFrame(rets)
df.head()

,name,address,type,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),starbucks,37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),starbucks,37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),starbucks,37.5139309,127.0206057
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),starbucks,37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),starbucks,37.494668,127.062583


이디아 커피 정보추출

In [80]:
url_2='https://www.ediya.com/contents/find_store.html'
driver.get(url_2)

input_xpath='//*[@id="storename"]'
input_box=driver.find_element(By.XPATH, input_xpath)
while input_box.is_displayed():
    try:
        loading_check=driver.find_element(By.XPATH, input_xpath)
        break
    except:
        time.sleep(0.1)
    


In [81]:
input_xpath='//*[@id="keyword"]'
ret_xpath='//*[@id="placesList"]/li'


action='//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a'
while True:
    try:
        action_btn=driver.find_element(By.XPATH, action)
        if action_btn.get_attribute('class')=='on':
            break
        else:
            action_btn.click()
    except:
        print('retrying...')
        time.sleep(1)


In [126]:
import re

rets_ediya=[]
input_box=driver.find_element(By.XPATH, input_xpath)
seoul_gu = ['강남구', '강동구', '강북구', '강서구',
             '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']

#rets_ediya=[]
for gu in seoul_gu:
    input_box.clear()
    input_box.send_keys(f"서울 {gu}")
    input_box.send_keys(Keys.RETURN)
    time.sleep(2)
    while True:
        try:
            store_list=driver.find_elements(By.XPATH, ret_xpath)
            if len(store_list)>0:
                break
        except:
            print('retrying...')
            time.sleep(5)
    for store in store_list:
        ret={}
        ret['name']=store.get_attribute('innerText').split('\n')[0]
        ret['address']=store.get_attribute('innerText').split('\n')[1]
        # latlng_string=store.find_element(By.XPATH, 'a').get_attribute('onclick')
        # pattern = r"panLatTo\('([-+]?\d*\.\d+|\d+)','([-+]?\d*\.\d+|\d+)','\d+'\);fnMove\(\);"
        
        # ## retrying until not 0
        # match = re.search(pattern, latlng_string)
        # ret['debug']=latlng_string
        # # while match is None:
        # #     print('retrying...')
        # #     latlng_string=store.find_element(By.XPATH, 'a').get_attribute('onclick')
        # #     time.sleep(0.5)
        # #     match = re.search(pattern, latlng_string)
        # try:
        #     ret['lat']=match.group(2)
        #     ret['lng']=match.group(1)
        #     if ret['lat']=='0':
        #         ret['lat']=None
        #     if ret['lng']=='0':
        #         ret['lng']=None
        # except:
        #     ret['lat']=None
        #     ret['lng']=None
        # ret['type']='ediya'
        rets_ediya.append(ret)

## update df
df=pd.DataFrame(rets_ediya)

에러발생한 케이스에 대해서 에러처리 (geocoding)

In [134]:
df

,name,address,debug,lat,lng,type
0,강남YMCA점,서울 강남구 논현동,"panLatTo('0','0','0');fnMove();",None,None,ediya
1,강남구청역아이티웨딩점,"서울 강남구 학동로 338 (논현동, 강남파라곤)","panLatTo('127.0401601992311','37.5165417172404...",37.51654171724045,127.0401601992311,ediya
2,강남논현학동점,서울 강남구 논현로131길 28 (논현동),"panLatTo('127.02810578707652','37.514080054467...",37.51408005446769,127.02810578707652,ediya
3,강남대치점,"서울 강남구 역삼로 415 (대치동, 성진빌딩)","panLatTo('127.05242928262568','37.501338761793...",37.50133876179308,127.05242928262568,ediya
4,강남도산점,서울 강남구 도산대로37길 20 (신사동),"panAddTo('서울 강남구 도산대로37길 20 (신사동)','4');fnMove();",None,None,ediya
...,...,...,...,...,...,...
719,중랑교차로점,서울 중랑구 동일로 683 (면목동),"panLatTo('0','0','25');fnMove();",None,None,ediya
720,중랑역점,서울 중랑구 망우로 198 (상봉동),"panAddTo('서울 중랑구 망우로 198 (상봉동)','26');fnMove();",None,None,ediya
721,중화동점,서울 중랑구 동일로129길 1 (중화동),"panLatTo('127.079542626772','37.5993838458016'...",37.5993838458016,127.079542626772,ediya
722,중화역점,"서울 중랑구 동일로 815, 1층","panAddTo('서울 중랑구 동일로 815, 1층','28');fnMove();",None,None,ediya


In [86]:
float(n)

0.0